In [19]:
import cv2
import numpy as np
import os
import random
from tqdm import tqdm
import argparse

In [20]:
def get_corners_from_yolo(bboxes, img_w, img_h):
    """Convert YOLO format [class, x_center, y_center, width, height] to pixel corner format [x1, y1, x2, y2]."""
    corners = []
    for bbox in bboxes:
        _, x_center, y_center, width, height = bbox
        w = width * img_w
        h = height * img_h
        x1 = (x_center * img_w) - (w / 2)
        y1 = (y_center * img_h) - (h / 2)
        x2 = x1 + w
        y2 = y1 + h
        corners.append([x1, y1, x2, y2])
    return np.array(corners)

In [21]:
def get_yolo_from_corners(corners, class_ids, img_w, img_h):
    """Convert pixel corner format [x1, y1, x2, y2] to YOLO format."""
    yolo_boxes = []
    for i, corner in enumerate(corners):
        x1, y1, x2, y2 = corner
        # Ensure x1 < x2 and y1 < y2
        x1, x2 = min(x1, x2), max(x1, x2)
        y1, y2 = min(y1, y2), max(y1, y2)
        
        # Clip to image boundaries
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(img_w, x2), min(img_h, y2)

        if x2 <= x1 or y2 <= y1:
            continue

        width = x2 - x1
        height = y2 - y1
        x_center = (x1 + width / 2) / img_w
        y_center = (y1 + height / 2) / img_h
        
        yolo_boxes.append([
            class_ids[i], 
            np.clip(x_center, 0, 1), 
            np.clip(y_center, 0, 1), 
            np.clip(width / img_w, 0, 1), 
            np.clip(height / img_h, 0, 1)
        ])
    return np.array(yolo_boxes)

In [22]:
def rotate_boxes(corners_4p, angle, cx, cy):
    """
    Rotate the 4 corners of each bounding box and return the new enclosing box.
    Args:
        corners_4p: (N, 4, 2) array of N boxes, each with 4 (x,y) corners.
        angle: rotation angle in degrees.
        cx, cy: rotation center.
    Returns:
        (N, 4) array of new enclosing bounding boxes in [x1, y1, x2, y2] format.
    """
    all_corners = corners_4p.reshape(-1, 2)
    all_corners_hom = np.hstack((all_corners, np.ones((all_corners.shape[0], 1))))

    M = cv2.getRotationMatrix2D((cx, cy), angle, 1.0)

    rotated_all_corners = np.dot(M, all_corners_hom.T).T
    rotated_corners_4p = rotated_all_corners.reshape(-1, 4, 2)

    x_min = np.min(rotated_corners_4p[:, :, 0], axis=1)
    y_min = np.min(rotated_corners_4p[:, :, 1], axis=1)
    x_max = np.max(rotated_corners_4p[:, :, 0], axis=1)
    y_max = np.max(rotated_corners_4p[:, :, 1], axis=1)

    return np.column_stack((x_min, y_min, x_max, y_max))

In [23]:
def augment_image(image, bboxes):
    h, w = image.shape[:2]
    class_ids = bboxes[:, 0].copy()
    augmentations = {}
    
    # 1. Horizontal Flip
    img_hf = cv2.flip(image, 1)
    bboxes_hf = bboxes.copy()
    bboxes_hf[:, 1] = 1 - bboxes_hf[:, 1]
    augmentations['hf'] = (img_hf, bboxes_hf)
    
    # 2. Brightness Adjustment
    brightness = random.uniform(0.7, 1.3)
    img_bright = cv2.convertScaleAbs(image, alpha=brightness, beta=0)
    augmentations['bright'] = (img_bright, bboxes.copy())
    
    # 3. Rotation
    angle = random.uniform(-10, 10)
    rot_mat = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
    img_rot = cv2.warpAffine(image, rot_mat, (w, h), borderMode=cv2.BORDER_REPLICATE)
    
    pixel_corners_2p = get_corners_from_yolo(bboxes, w, h)
    if pixel_corners_2p.size == 0:
        return augmentations # Return other augmentations if no boxes

    x1 = pixel_corners_2p[:, 0:1]
    y1 = pixel_corners_2p[:, 1:2]
    x2 = pixel_corners_2p[:, 2:3]
    y2 = pixel_corners_2p[:, 3:4]
    
    corners_4p = np.hstack([x1, y1, x2, y1, x1, y2, x2, y2]).reshape(-1, 4, 2)

    rotated_pixel_corners = rotate_boxes(corners_4p, angle, w/2, h/2)
    
    bboxes_rot = get_yolo_from_corners(rotated_pixel_corners, class_ids, w, h)
    if bboxes_rot.size > 0:
        augmentations['rot'] = (img_rot, bboxes_rot)

    return augmentations

In [24]:
def main(base_dir):
    train_dir = os.path.join(base_dir, 'train')
    image_dir = os.path.join(train_dir, 'images')
    label_dir = os.path.join(train_dir, 'labels')

    if not os.path.exists(image_dir) or not os.path.exists(label_dir):
        print(f"Error: Training directories not found in {train_dir}")
        return

    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png')) and not f.startswith('aug_')]
    print(f'Found {len(image_files)} original images to augment.')

    for image_name in tqdm(image_files, desc='Augmenting Images'):
        base_name, ext = os.path.splitext(image_name)
        label_name = f'{base_name}.txt'
        image_path = os.path.join(image_dir, image_name)
        label_path = os.path.join(label_dir, label_name)

        if not os.path.exists(label_path):
            continue
            
        image = cv2.imread(image_path)
        if image is None:
            continue
            
        with open(label_path, 'r') as f:
            try:
                bboxes = np.array([line.strip().split() for line in f.readlines()], dtype=np.float32)
            except ValueError:
                continue # Skip malformed label files
            
        if bboxes.size == 0 or bboxes.shape[1] != 5:
            continue
            
        augmented_data = augment_image(image, bboxes)
        
        for aug_type, (aug_img, aug_bboxes) in augmented_data.items():
            if aug_bboxes.size == 0:
                continue
            aug_img_name = f'aug_{base_name}_{aug_type}{ext}'
            aug_label_name = f'aug_{base_name}_{aug_type}.txt'
            cv2.imwrite(os.path.join(image_dir, aug_img_name), aug_img)
            np.savetxt(os.path.join(label_dir, aug_label_name), aug_bboxes, fmt='%.6f')

    print('\nData augmentation complete!')
    print(f'Total images in training set now: {len(os.listdir(image_dir))}')

In [25]:
if __name__ == '__main__':
    default_path = 'C:/Empty_Parking/PKLot.v2-640.yolov8'
    main(default_path)

Found 8691 original images to augment.


Augmenting Images: 100%|███████████████████████████████████████████████████████████| 8691/8691 [01:28<00:00, 97.66it/s]


Data augmentation complete!
Total images in training set now: 34197
